In [3]:
import sys
sys.path.insert(0, "../..")

In [4]:
import torch
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import numpy as np
from copy import deepcopy

from nn_extrapolation import AcceleratedSGD, SeparateAcceleratedSGD

In [5]:
torch.cuda.is_available()

True

In [6]:
val_loss_fn = nn.NLLLoss(reduction="sum")

def validation(model, loader):
    ok = 0
    loss_sum = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.cuda()
            y = y.cuda()
            out = model(x)
            loss_sum += val_loss_fn(out, y)
            preds = out.argmax(1)
            ok += (y == preds).sum()
            total += len(y)
    return ok / total, loss_sum / total

def train_epoch():
    model.train()
    param_hist = []
    for i, (x, y) in enumerate(tqdm(train_loader)):
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()
    return torch.tensor(np.mean(param_hist, 0))

In [7]:
train_ds = datasets.MNIST("../../../MNIST", download=True, train=True, transform=transforms.ToTensor())
test_ds = datasets.MNIST("../../../MNIST", download=True, train=False, transform=transforms.ToTensor())
valid_size = int(0.2 * len(train_ds))
train_ds, valid_ds = data.random_split(train_ds, [len(train_ds) - valid_size, valid_size])

train_loader = data.DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)
valid_loader = data.DataLoader(valid_ds, batch_size=64, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=2)

In [8]:
model = nn.Sequential(
    nn.Conv2d(1, 32, 3),
    nn.ReLU(),
    nn.Conv2d(32, 32, 3),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 3),
    nn.ReLU(),
    nn.Conv2d(64, 64, 3),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(4*4*64, 128),
    nn.ReLU(),
    nn.Linear(128, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [9]:
initial_state = deepcopy(model.state_dict())

In [10]:
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, mode="epoch", method="RRE")
loss_fn = nn.NLLLoss()

## Epoch

In [11]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1


/opt/miniconda3/envs/torch/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/miniconda3/envs/torch/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Validation accuracy: 0.1005, validation loss: 2.3013
Epoch 2



Validation accuracy: 0.1005, validation loss: 2.3007
Epoch 3



Validation accuracy: 0.1450, validation loss: 2.3000
Epoch 4


...


Validation accuracy: 0.9387, validation loss: 0.1930
Epoch 27



Validation accuracy: 0.9493, validation loss: 0.1710
Epoch 28



Validation accuracy: 0.9492, validation loss: 0.1657
Epoch 29



Validation accuracy: 0.9518, validation loss: 0.1623
Epoch 30



Validation accuracy: 0.9548, validation loss: 0.1441


In [12]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9587, device='cuda:0'), tensor(0.1371, device='cuda:0'))



Valid: (tensor(0.9548, device='cuda:0'), tensor(0.1441, device='cuda:0'))


In [13]:
optimizer.accelerate()

In [14]:
optimizer.store_parameters()

In [15]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9597, device='cuda:0'), tensor(0.1311, device='cuda:0'))



Valid: (tensor(0.9565, device='cuda:0'), tensor(0.1383, device='cuda:0'))


## Epoch average

In [17]:
model.load_state_dict(initial_state)
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, mode="epoch_avg", method="RRE")

In [18]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1



Validation accuracy: 0.1005, validation loss: 2.3013
Epoch 2



Validation accuracy: 0.1005, validation loss: 2.3007
Epoch 3



Validation accuracy: 0.1435, validation loss: 2.3000
Epoch 4



Validation accuracy: 0.1131, validation loss: 2.2993
Epoch 5


...


Validation accuracy: 0.9426, validation loss: 0.1901
Epoch 27



Validation accuracy: 0.9462, validation loss: 0.1738
Epoch 28



Validation accuracy: 0.9463, validation loss: 0.1759
Epoch 29



Validation accuracy: 0.9523, validation loss: 0.1573
Epoch 30



Validation accuracy: 0.9567, validation loss: 0.1429


In [19]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9585, device='cuda:0'), tensor(0.1365, device='cuda:0'))



Valid: (tensor(0.9567, device='cuda:0'), tensor(0.1429, device='cuda:0'))


In [20]:
optimizer.accelerate()

In [21]:
optimizer.store_parameters()

In [22]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9649, device='cuda:0'), tensor(0.1191, device='cuda:0'))



Valid: (tensor(0.9605, device='cuda:0'), tensor(0.1262, device='cuda:0'))
